In [1]:
# Dependencies
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

from config_z import (gkey)

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [2]:
#File to Load (Remember to Change These)
file_to_load = "IRS_Data_2015_CSV.csv"
file_to_load_2 = "MplsZipcodes.csv"

#Read Purchasing File and store into Pandas data frame
IRS_data = pd.read_csv(file_to_load)
Zip_data = pd.read_csv(file_to_load_2)

In [3]:
IRS_data = pd.read_csv('IRS_Data_2015_CSV.csv' , usecols = ['Zip codes','-1','-17'])

In [4]:
IRS_data = IRS_data.dropna()

In [5]:
IRS_data_fliterd = IRS_data.drop_duplicates('Zip codes')

In [6]:
IRS_data_cleaned = IRS_data_fliterd.rename (columns={"-1":"Number of returns","-17":"Total income amount"})
IRS_data_cleaned["Zip codes"] = (IRS_data_cleaned["Zip codes"]).astype(int)

In [8]:
combined_zip_codes = pd.merge(Zip_data, IRS_data_cleaned, on="Zip codes")
combined_zip_codes

,Zip codes,Lat,Long,Number of returns,Total income amount
0,55401,44.984577,-93.269097,"6,560","951,609"
1,55402,44.976039,-93.271444,860,"565,582"
2,55403,44.970570,-93.284044,"9,970","1,006,822"
3,55404,44.961996,-93.261208,"12,310","467,957"
4,55405,44.972669,-93.301675,"8,650","994,104"
5,55406,44.938689,-93.221042,"18,550","1,158,199"
6,55407,44.935036,-93.253029,"18,890","868,889"
7,55408,44.940425,-93.292639,"18,840","1,076,675"
8,55409,44.930326,-93.281477,"6,380","472,111"
9,55410,44.911823,-93.319643,"10,360","1,458,333"


In [ ]:
csv_path = "Police_Incidents_2015.csv"
police_data = pd.read_csv(csv_path)

In [ ]:
theft_df = police_data.loc[police_data["Description"] == "Other Theft",:]
burg_dwell_df = police_data.loc[police_data["Description"] == "Burglary Of Dwelling",:]
tfmv_df = police_data.loc[police_data["Description"] == "Theft From Motr Vehc",:]
bike_theft = police_data.loc[police_data["Description"] == "Bike Theft",:]

In [ ]:
params = {"key":gkey}

for index, row in theft_df.iterrows():
    lat = row['Lat']
    lng = row ['Long']
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    zip_lat_long = f"{base_url}latlng={lat},{lng}&key={gkey}"
    zip_code = requests.get(zip_lat_long)
    zip_code = zip_code.json()
    try:
         theft_df.loc[index,"Zip Code"] = zip_code['results'][0]['address_components'][7]['long_name']
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")


In [ ]:
params = {"key":gkey}

for index, row in burg_dwell_df.iterrows():
    lat = row['Lat']
    lng = row ['Long']
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    zip_lat_long = f"{base_url}latlng={lat},{lng}&key={gkey}"
    zip_code = requests.get(zip_lat_long)
    zip_code = zip_code.json()
    try:
         burg_dwell_df.loc[index,"Zip Code"] = zip_code['results'][0]['address_components'][7]['long_name']
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")


In [ ]:
params = {"key":gkey}

for index, row in tfmv_df.iterrows():
    lat = row['Lat']
    lng = row ['Long']
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    zip_lat_long = f"{base_url}latlng={lat},{lng}&key={gkey}"
    zip_code = requests.get(zip_lat_long)
    zip_code = zip_code.json()
    try:
         tfmv_df.loc[index,"Zip Code"] = zip_code['results'][0]['address_components'][7]['long_name']
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")

In [ ]:
params = {"key":gkey}

for index, row in bike_theft.iterrows():
    lat = row['Lat']
    lng = row ['Long']
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    zip_lat_long = f"{base_url}latlng={lat},{lng}&key={gkey}"
    zip_code = requests.get(zip_lat_long)
    zip_code = zip_code.json()
    try:
         bike_theft.loc[index,"Zip Code"] = zip_code['results'][0]['address_components'][7]['long_name']
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")


In [ ]:
theft_df = theft_df.dropna()
tfmv_df.dropna()
bike_theft = bike_theft.dropna()

In [ ]:
theft_df.to_csv("theft_df.csv")

In [ ]:
burg_dwell_df.to_csv("burg_dwell_df.csv")

In [ ]:
tfmv_df.to_csv('tfmv_df.csv')

In [ ]:
# Store 'Lat' and 'Lng' into  locations for zip codes
zip_locations = combined_zip_codes[["Lat", "Long"]].astype(float)

In [ ]:
# Store 'Lat' and 'Lng' into  locations for theft
locations_theft = theft_df[["Lat", "Long"]].astype(float)

In [ ]:
# Store 'Lat' and 'Lng' into  locations for burgulary
locations_burg = burg_dwell_df[["Lat", "Long"]].astype(float)

In [ ]:
# Store 'Lat' and 'Lng' into  locations for theft from motor vehicle
locations_tfmv = tfmv_df[["Lat", "Long"]].astype(float)

In [ ]:
# Store 'Lat' and 'Lng' into  locations for theft from motor vehicle
locations_bike = bike_theft[["Lat", "Long"]].astype(float)

In [ ]:
# Create bank symbol layer
theft_dots = gmaps.symbol_layer(
    locations_theft, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 250, 0.4)', scale=1,)

fig = gmaps.figure()
fig.add_layer(theft_dots)
fig

In [ ]:
# Create bank symbol layer
burg_dots = gmaps.symbol_layer(
    locations_burg, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(150, 0, 0, 0.4)', scale=1,)

fig = gmaps.figure()
fig.add_layer(burg_dots)
fig

In [ ]:
# Create bank symbol layer
tfmv_dots = gmaps.symbol_layer(
    locations_tfmv, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=1,)

fig = gmaps.figure()
fig.add_layer(tfmv_dots)
fig

In [ ]:
# Create bank symbol layer
bike_dots = gmaps.symbol_layer(
    locations_bike, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 1)', scale=2,)

fig = gmaps.figure()
fig.add_layer(bike_dots)
fig

In [ ]:
zip_dots = gmaps.symbol_layer(
    zip_locations, fill_color='rgba(500, 0, 0, 0.4)',
    stroke_color='rgba(500, 0, 0, 0.4)', scale=2,)

fig = gmaps.figure()
fig.add_layer(zip_dots)
fig.add_layer(bike_dots)
fig